## Build a Brochure for company with AI

In [1]:
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from google import genai

In [2]:
load_dotenv(override=True)
api_key = os.getenv('GOOGLE_API_KEY')

   
if api_key:
    print("Successfully loaded the API key from .env file!")
else:
    print("API key found and looks good so far!")

Successfully loaded the API key from .env file!


In [3]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddo

In [12]:
from google import genai
from google.genai import types


client = genai.Client()

def get_links(url):
    website = Website(url)
    response =  client.models.generate_content(
        model="gemini-2.5-flash",
        config=types.GenerateContentConfig(
        system_instruction=link_system_prompt),
        contents=get_links_user_prompt(website)
    )
    result = response.text
    return result

In [13]:
huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/black-forest-labs/FLUX.1-Kontext-dev',
 '/tencent/Hunyuan-A13B-Instruct',
 '/google/magenta-realtime',
 '/google/gemma-3n-E4B-it',
 '/nanonets/Nanonets-OCR-s',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/OmniGen2/OmniGen2',
 '/spaces/ilcve21/Sparc3D',
 '/spaces/black-forest-labs/FLUX.1-Kontext-Dev',
 '/spaces/tencent/Hunyuan3D-2.1',
 '/spaces',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/institutional/institutional-books-1.0',
 '/datasets/facebook/seamless-interaction',
 '/datasets/EssentialAI/essential-web-v1.0',
 '/datasets/FreedomIntelligence/ShareGPT-4o-Image',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammar

In [14]:
get_links("https://huggingface.co")

'```json\n{\n    "links": [\n        {"type": "homepage", "url": "https://huggingface.co/"},\n        {"type": "company profile", "url": "https://huggingface.co/huggingface"},\n        {"type": "careers page", "url": "https://apply.workable.com/huggingface/"},\n        {"type": "enterprise solutions", "url": "https://huggingface.co/enterprise"},\n        {"type": "pricing information", "url": "https://huggingface.co/pricing"},\n        {"type": "blog", "url": "https://huggingface.co/blog"},\n        {"type": "learning resources", "url": "https://huggingface.co/learn"},\n        {"type": "models directory", "url": "https://huggingface.co/models"},\n        {"type": "datasets directory", "url": "https://huggingface.co/datasets"},\n        {"type": "spaces directory", "url": "https://huggingface.co/spaces"},\n        {"type": "github profile", "url": "https://github.com/huggingface"},\n        {"type": "twitter profile", "url": "https://twitter.com/huggingface"},\n        {"type": "linked